In [25]:
import datetime as dt
import math
import sys

import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
plt.rcParams.update({'figure.max_open_warning': 0})


import numpy as np
import pandas as pd
import tweepy
from matplotlib import style
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler, StandardScaler, Normalizer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import pickle
import plotnine as p9

from textblob import TextBlob

# import constants as ct
# from Tweet import Tweet

import src.functions as DLmodels

style.use('ggplot')

stocks = pd.read_csv('bovespa.csv')
stocks = stocks['codigo'].values


n_steps_in, n_steps_out = 30, 3
interval = '1d'
for stock in stocks:


    size = len(DLmodels.get_stock_data(stock, '1d'))
    data = pd.DataFrame()
    for ML in ['LSTM', 'BidirectionalLSTM', 'ConvLSTM1D', 'ConvLSTM2D']:

        dataset = pd.read_csv('logs/FG-' + stock + '-' + ML + '-' + interval + '-' + str(n_steps_in) +'-'+ str(n_steps_out) +'.log')

        df_loss = dataset[['epoch', 'loss']]
        df_loss['metric'] = 'loss'            
        df_loss.columns = ['epoch', 'value', 'metric']            

        df_val_loss = dataset[['epoch', 'val_loss']]
        df_val_loss['metric'] = 'val_loss'
        df_val_loss.columns = ['epoch', 'value', 'metric']

        dataset = pd.concat([df_loss, df_val_loss])

        dataset['ML_tecnique'] = ML
        data = pd.concat([data, dataset])

    plot_describe_groupedData = p9.ggplot(data=data) +\
        p9.geom_line(mapping=p9.aes(x="epoch", y="value", color='metric', group='metric')) +\
        p9.scales.scale_y_log10() +\
        p9.facet_wrap('ML_tecnique', scales="fixed") +\
        p9.theme(axis_text_x = p9.element_text(angle=90))    +\
        p9.labs(title='Size of the data: ' + str(size))


    plot_describe_groupedData.save(filename= 'images/logs_FG-' + stock + '-' + str(n_steps_in) +'-'+ str(n_steps_out) + '.png')


/home/marcos/tf-cpu/lib/python3.8/site-packages/plotnine/ggplot.py:727: PlotnineWarning: Saving 6.4 x 4.8 in image.
/home/marcos/tf-cpu/lib/python3.8/site-packages/plotnine/ggplot.py:730: PlotnineWarning: Filename: images/logs_FG-ABCB4-30-3.png
/home/marcos/tf-cpu/lib/python3.8/site-packages/plotnine/ggplot.py:727: PlotnineWarning: Saving 6.4 x 4.8 in image.
/home/marcos/tf-cpu/lib/python3.8/site-packages/plotnine/ggplot.py:730: PlotnineWarning: Filename: images/logs_FG-ABEV3-30-3.png
/home/marcos/tf-cpu/lib/python3.8/site-packages/plotnine/ggplot.py:727: PlotnineWarning: Saving 6.4 x 4.8 in image.
/home/marcos/tf-cpu/lib/python3.8/site-packages/plotnine/ggplot.py:730: PlotnineWarning: Filename: images/logs_FG-ANIM3-30-3.png
/home/marcos/tf-cpu/lib/python3.8/site-packages/plotnine/ggplot.py:727: PlotnineWarning: Saving 6.4 x 4.8 in image.
/home/marcos/tf-cpu/lib/python3.8/site-packages/plotnine/ggplot.py:730: PlotnineWarning: Filename: images/logs_FG-AGRO3-30-3.png
/home/marcos/tf-cpu/

/home/marcos/tf-cpu/lib/python3.8/site-packages/plotnine/ggplot.py:727: PlotnineWarning: Saving 6.4 x 4.8 in image.
/home/marcos/tf-cpu/lib/python3.8/site-packages/plotnine/ggplot.py:730: PlotnineWarning: Filename: images/logs_FG-CRFB3-30-3.png
/home/marcos/tf-cpu/lib/python3.8/site-packages/plotnine/ggplot.py:727: PlotnineWarning: Saving 6.4 x 4.8 in image.
/home/marcos/tf-cpu/lib/python3.8/site-packages/plotnine/ggplot.py:730: PlotnineWarning: Filename: images/logs_FG-CSMG3-30-3.png
/home/marcos/tf-cpu/lib/python3.8/site-packages/plotnine/ggplot.py:727: PlotnineWarning: Saving 6.4 x 4.8 in image.
/home/marcos/tf-cpu/lib/python3.8/site-packages/plotnine/ggplot.py:730: PlotnineWarning: Filename: images/logs_FG-CSAN3-30-3.png
/home/marcos/tf-cpu/lib/python3.8/site-packages/plotnine/ggplot.py:727: PlotnineWarning: Saving 6.4 x 4.8 in image.
/home/marcos/tf-cpu/lib/python3.8/site-packages/plotnine/ggplot.py:730: PlotnineWarning: Filename: images/logs_FG-CSNA3-30-3.png
/home/marcos/tf-cpu/

FileNotFoundError: [Errno 2] File logs/FG-FLRY3-ConvLSTM1D-1d-30-3.log does not exist: 'logs/FG-FLRY3-ConvLSTM1D-1d-30-3.log'

In [36]:
import sys

import numpy as np
import pandas as pd
import src.constants_stocks as cs
from sklearn.preprocessing import MinMaxScaler, StandardScaler

import pickle
import src.functions as DLmodels


n_steps_in, n_steps_out = 3, 3

samples_test = 1

ML_Techniques = ['LSTM', 'BidirectionalLSTM', 'convLSTM1D', 'convLSTM2D']     
interval='1d'

datagrouped = DLmodels.data_grouped_foreign_stock(cs.foreign_stocks, interval)

all_MAPE = []
for stock in cs.stocks_codigo[0:5]:
    print(stock)
    (flag, symbol) = (True, stock)
    if flag:
        
        dataframe = DLmodels.get_stock_data(symbol, interval)
        
        df2 = dataframe[['Volume', 'Adj Close']]       
        df2.index = dataframe['Date']
        dataframe = pd.merge(datagrouped,df2, how='inner', left_index=True, right_index=True)
        
        dataframe = DLmodels.clean_dataset(dataframe)

        dataframe = dataframe.dropna()
        
        scaler = StandardScaler()
        dataset = scaler.fit_transform(dataframe.ffill().values)

        scaler_filename = 'scalers/' + stock + '-' + interval + '-FG.save'
        scaler = pickle.load(open(scaler_filename, 'rb'))
        dataset = scaler.fit_transform(dataframe.iloc[:,:].ffill().values)
        
        n_features = dataset.shape[1]  
        
        X = np.reshape(dataset[-n_steps_in:, :], (1,n_steps_in,n_features))
            
              
        y = dataset[-n_steps_out:,-1:]
        
        for ML_tech in ML_Techniques: 
           
            if ML_tech == 'lr':
                (predictions, accuracy) = DLmodels.stock_forecasting(dataframe, n_steps_out)
            if ML_tech == 'lrLog':
                (predictions, accuracy) = DLmodels.stock_forecastingLog(dataframe, n_steps_out)                
            if ML_tech == 'conv':
                predictions = DLmodels.predict_conv1D(stock, np.reshape(X[-samples_test:], (samples_test, n_steps_in, n_features)), interval, n_steps_in)
            if ML_tech == 'LSTM':
                predictions = DLmodels.predict_LSTM('FG-' + stock, np.reshape(X[-samples_test:], (samples_test,n_steps_in, n_features)), interval, n_steps_in)
            if ML_tech == 'BidirectionalLSTM':
                predictions = DLmodels.predict_BidirectionalLSTM('FG-' + stock, np.reshape(X[-samples_test:], (samples_test,n_steps_in, n_features)), interval, n_steps_in)
            if ML_tech == 'convLSTM1D':
                predictions = DLmodels.predict_convLSTM1D('FG-' + stock, np.reshape(X[-samples_test:], (samples_test,n_steps_in, n_features, 1)), interval, n_steps_in)
            if ML_tech == 'convLSTM2D':
                predictions = DLmodels.predict_convLSTM2D('FG-' + stock, np.reshape(X[-samples_test:], (samples_test,n_steps_in, n_features, 1, 1)), interval, n_steps_in)           
            
            MAPE = DLmodels.mean_absolute_percentage_error(np.reshape(y[-samples_test:], (samples_test*n_steps_out,)), np.reshape(predictions, (samples_test*n_steps_out,)))
            
            predictions = np.concatenate([np.reshape(y[-1:], (1,)), np.reshape(predictions, (n_steps_out,))], axis = 0)
       
    
            (gain_pred, index_maximo_pred) = DLmodels.predictions_max_index(predictions, n_steps_out)
        
            all_MAPE.append([stock, ML_tech, np.around(MAPE,1), gain_pred, index_maximo_pred, n_steps_in, n_steps_out, interval, samples_test])
        
dataFrame = pd.DataFrame(all_MAPE)

dataFrame.columns = ['Stock', 'ML', 'MAPE', 'gain_pred', 'index_maximo_pred', 'n_steps_in', 'n_steps_out', 'interval', 'samples_test']          
          
dataFrame.to_csv('logs/data-future_mix.csv')



ABCB4


ValueError: cannot reshape array of size 1 into shape (3,)

In [43]:
y

array([[0.42810723],
       [0.47441105],
       [0.43541836]])

In [16]:
DLmodels.predict_LSTM('FG-' + stock, np.reshape(X[-samples_test:], (samples_test,n_steps_in, n_features)), interval, n_steps_in)

ValueError: in user code:

    /home/marcos/tf-cpu/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1462 predict_function  *
        return step_function(self, iterator)
    /home/marcos/tf-cpu/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1452 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/marcos/tf-cpu/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/marcos/tf-cpu/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/marcos/tf-cpu/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/marcos/tf-cpu/lib/python3.8/site-packages/tensorflow/python/keras/eng

In [76]:
import nltk
import tweepy
import constants as ct
from Tweet import Tweet
from textblob import TextBlob

actual_date = dt.date.today()
past_date = actual_date - dt.timedelta(days=2)
date_since = past_date.strftime("%Y-%m-%d")

symbol = 'Amazon'
# nltk.download('punkt')
auth = tweepy.OAuthHandler(ct.consumer_key, ct.consumer_secret)
auth.set_access_token(ct.access_token, ct.access_token_secret)
user = tweepy.API(auth, wait_on_rate_limit=True)

tweets = user.search(symbol)
tweet_list = []
global_polarity = 0
polarity = 0
for tweet in tweets:
    tw = tweet.text
    blob = TextBlob(tw)
    print(blob.sentiment)

Sentiment(polarity=-0.1, subjectivity=0.1)
Sentiment(polarity=0.0, subjectivity=0.0)
Sentiment(polarity=0.0, subjectivity=0.0)
Sentiment(polarity=0.0, subjectivity=0.0)
Sentiment(polarity=-0.4333333333333333, subjectivity=0.6)
Sentiment(polarity=0.0, subjectivity=0.0)
Sentiment(polarity=0.0, subjectivity=0.0)
Sentiment(polarity=0.0, subjectivity=0.0)
Sentiment(polarity=0.0, subjectivity=0.0)
Sentiment(polarity=0.25, subjectivity=0.5)
Sentiment(polarity=0.0, subjectivity=0.0)
Sentiment(polarity=0.0, subjectivity=0.0)
Sentiment(polarity=0.0, subjectivity=0.0)
Sentiment(polarity=0.0, subjectivity=0.0)
Sentiment(polarity=0.0, subjectivity=0.0)


Sentiment(polarity=0.0, subjectivity=0.0)
Sentiment(polarity=0.0, subjectivity=0.0)
Sentiment(polarity=0.0, subjectivity=0.0)
Sentiment(polarity=-0.025, subjectivity=0.35)
Sentiment(polarity=0.0, subjectivity=0.0)
Sentiment(polarity=0.0, subjectivity=0.0)
Sentiment(polarity=0.0, subjectivity=0.0)
Sentiment(polarity=0.0, subjectivity=0.0)
Sentiment(polarity=0.0, subjectivity=0.0)
Sentiment(polarity=0.0, subjectivity=0.0)
Sentiment(polarity=0.0, subjectivity=0.0)
Sentiment(polarity=0.0, subjectivity=0.0)
Sentiment(polarity=0.0, subjectivity=0.0)
Sentiment(polarity=0.0, subjectivity=0.0)
Sentiment(polarity=0.0, subjectivity=0.0)


In [61]:
for tweet in tweets:
    tw = tweet.full_text
    blob = TextBlob(tw)
    print(blob)

RT @EduardoPFreme: @investidorIn Eu deveria ter silenciado COGN3 em junho...
Cogna (#COGN3): cenário permanece desafiador, mas negócio com Eleva pode acelerar crescimento.

https://t.co/dX1yGBCCVO
@m_ichel_s @ZattarRafael O que aconteceu com COGN3 hoje? Kkk Subiu 7% e devolveu tudo.
@investidorIn Eu deveria ter silenciado COGN3 em junho...
RT @ossoasset: Nesse fio polêmico, enquanto os clubes e latas de sardinhas defendiam apaixonadamente as teses de COGN3 e IRBR3, uma persona…
Nesse fio polêmico, enquanto os clubes e latas de sardinhas defendiam apaixonadamente as teses de COGN3 e IRBR3, uma personagem passou despercebida pelo radar de muitos: VALE3. Da Squadra não. Vc conferiu esses 3 racionais da carta, na época, e comparou com o outcome? Pois é. https://t.co/rPFBG1jTtg
RT @Carlos_Trumpp: Cogna (COGN3) negocia com a Eleva Educação, PicPay pronto para seu IPO

https://t.co/SwgxJvxXGd
Cogna (COGN3) negocia com a Eleva Educação, PicPay pronto para seu IPO

https://t.co/SwgxJvxXGd
RT @b

RT @deuruimpessoal: Já dá para colocar 10% do PL em call seca de COGN3 @nonoinvestidor?
Já dá para colocar 10% do PL em call seca de COGN3 @nonoinvestidor?
RT @Diego44297619: Cogn3 subindo por causa de notícia, q pode ser positiva mais nem se concretizou , aí bando de PF sai comprando opção a s…
Cogna (COGN3) mais de 4% após confirmação de negociação de ativos com Eleva https://t.co/yqXsMl95tq
A Cogna detalha que a transação poderá envolver a venda de determinadas escolas controladas direta ou indiretamente pela Saber à Eleva.  #COGN3 #Negócios

https://t.co/tq2kMmMseF
Cogn3 subindo por causa de notícia, q pode ser positiva mais nem se concretizou , aí bando de PF sai comprando opção a seco , a vol pra alturas e vendem na cabeça dos sardinhas. Operar notícia fracasso certo. Tem q operar o negócio
‼️ As 5 Maiores Altas do #Ibovespa na Abertura – 07/01 - 11:37hr

📈 #SUZB3  +6,00% (R$60,97) 
📈 #USIM5 +4,90% (R$16,49) 
📈 #KLBN11 +4,67% (R$27,36)
📈 #COGN3 +4,42% (R$4,72) 
📈 #CSNA3 +4,17% (R

RT @FilipeVillegas: 📊 Mais negociadas investidor PF 🐟🐡🐠 (Lote #Fracionário)

Ticker / Var(%) / N° de Ações

#MGLU3 / -5,2% / 419K
#OIBR3 /…
RT @FilipeVillegas: 📊 Ações +Termadas #termos 🦈

🧐 Apostando numa alta ou defendendo posição

Ticker / Var(%) / N° de Ações

#OIBR3 / -1,2%…
📊 Ranking posições ATUAL 🆚 pregão ANTERIOR +Termadas

#OIBR3 1° ⬆️ 3°
#JBSS3 2° ⏸ 2°
#IRBR3 3° ⬆️ 4°
#VALE3 4° ⬆️ 
#BRFS3 5° ⬆️ 8°
#BRKM5 6° ⬆️ 
#PETR4 7° ⬆️ 
#COGN3 8° ⬇️ 1°
#VVAR3 9° ⬇️ 10°
#BBDC4 10° ⬆️ 

#marketmovers
📊 Ações +Termadas #termos 🦈

🧐 Apostando numa alta ou defendendo posição

Ticker / Var(%) / N° de Ações

#OIBR3 / -1,2% / 3.501K
#JBSS3 / -0,6% / 1.126K
#IRBR3 / -0,1% / 921K
#VALE3 / +2,8% / 841K
#BRFS3 / -0,5% / 760K

Além disso tivemos #BRKM5 #PETR4 #COGN3 #VVAR3 e #BBDC4
📊 Ranking posições ATUAL 🆚 pregão ANTERIOR 🐟🐡🐠 (Lote Fracionário)

#MGLU3 1° ⬆️ 4°
#OIBR3 2° ⬇️ 1°
#VVAR3 3° ⏸ 3°
#WEGE3 4° ⬆️ 6°
#ITSA4 5° ⬇️ 2°
#PETR4 6° ⬇️ 5°
#BBDC4 7° ⬆️ 9°
#B3SA3 8° ⬆️ 
#COGN3 9° ⬆️ 
#EMBR3 10° ⬆️ 


In [4]:
import sys

import numpy as np
import pandas as pd
          

    
dados = pd.read_csv('./logs/dados-MAPE.csv')

dados


dados['Gain'].loc[dados['Gain'] > 10] = 10
dados = dados.loc[(dados['Gain'] > 0) & (dados['TSA'] > 0)]
filenamePlot = '-self'
var_plot_bar_all_predictions = DLmodels.plot_bar_all_predictions(dados, filenamePlot)



,Unnamed: 0,Name,ML,MAPE,n_steps_in,n_steps_out,interval,samples_test
0,0,ABCB4,conv,14.6,5,1,1wk,5
1,1,ABCB4,LSTM,62.8,5,1,1wk,5
2,2,ABCB4,BidirectionalLSTM,23.8,5,1,1wk,5
3,3,ABCB4,convLSTM1D,34.7,5,1,1wk,5
4,4,ABCB4,convLSTM2D,68.6,5,1,1wk,5
5,5,ABEV3,conv,22.2,5,1,1wk,5
6,6,ABEV3,LSTM,27.0,5,1,1wk,5
7,7,ABEV3,BidirectionalLSTM,31.1,5,1,1wk,5
8,8,ABEV3,convLSTM1D,18.4,5,1,1wk,5
9,9,ABEV3,convLSTM2D,40.5,5,1,1wk,5
